In [1]:
import cv2
import os
import time
import pandas as pd
import pyodbc
import datetime
import json

In [3]:
dataPath = '/home/nvidia/Documents/RF/Reconocimiento Facial/Data' #Cambia a la ruta donde hayas almacenado Data
imagePaths = sorted(os.listdir(dataPath))
print('imagePaths=',imagePaths)

#face_recognizer = cv2.face.EigenFaceRecognizer_create()
#face_recognizer = cv2.face.FisherFaceRecognizer_create()
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Leyendo el modelo
#face_recognizer.read('modeloEigenFace.xml')
#face_recognizer.read('modeloFisherFace.xml')
face_recognizer.read(dataPath + 'modeloLBPHFace.xml')

cap = cv2.VideoCapture(1)
#cap = cv2.VideoCapture('Video.mp4')

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

# Try connect
cred =  open('cred.json',)
credentials = json.load(cred)
server = credentials['raiz']['server']
database = credentials['raiz']['database']
username = credentials['raiz']['username']
passw = credentials['raiz']['password']
driver= '/usr/lib/aarch64-linux-gnu/odbc/libtdsodbc.so'
table_name = 'RegistroAsistencia'

cnxn = pyodbc.connect(
    driver = driver,
    TDS_Version = '7.3', 
    server = server,
    port = 1433,
    DATABASE= 'AsistenciabSide',
    uid = username,
    pwd = passw)
cursor = cnxn.cursor()
registro = {'id_empleado': [], 'entrada':[]}

while True:
	ret,frame = cap.read()
	if ret == False: break
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	auxFrame = gray.copy()

	faces = faceClassif.detectMultiScale(gray,1.3,5)

	for (x,y,w,h) in faces:
		rostro = auxFrame[y:y+h,x:x+w]
		rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
		result = face_recognizer.predict(rostro)

		cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
		'''
		# EigenFaces
		if result[1] < 5700:
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
		else:
			cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
		
		# FisherFace
		if result[1] < 500:
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
		else:
			cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
		'''

		# LBPHFace
		if result[1] < 70 :
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
			
			label = imagePaths[result[0]]
			if label in registro['id_empleado']:
				pass
			else:
				registro['id_empleado'].append(label)
				registro['entrada'].append(datetime.datetime.now())
				dfasis = pd.DataFrame.from_dict(registro)
				dfasis['entrada'] = pd.to_datetime(dfasis['entrada'])
				# Insert Dataframe into SQL Server:
				time.sleep(1)
				for index, row in dfasis.iterrows():
					cursor.execute("INSERT INTO dbo.RegistroAsistencia (id_empleado,entrada) values(?,?)", row.id_empleado, row.entrada)
				cnxn.commit()
				time.sleep(1)
		else:
			cv2.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)

	cv2.imshow('frame',frame)
	k = cv2.waitKey(30)
	if  k & 0xff == ord('q'):
		cursor.close()
		break

cap.release()
cv2.destroyAllWindows()

imagePaths= ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '59', '6', '63', '64', '65', '7', '8', '9']
